In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
from otc.metrics.metrics import effective_spread

import wandb

from tqdm.notebook import tqdm

from scipy.stats import wilcoxon

In [2]:
# set here globally
exchange = "ise"
models = ["gbm", "classical"]
subset = "test" # "all"
strategy = "supervised"

key = f"{exchange}_all_{strategy}_{subset}"

In [12]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

dataset = f"fbv/thesis/{exchange}_{strategy}_raw:latest"

# load unscaled data
artifact = run.use_artifact(dataset) # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in models:
    results = f"fbv/thesis/{exchange}_{model}_{strategy}_{subset}:latest"
    artifact = run.use_artifact(results) # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


wandb: Downloading large artifact ise_supervised_raw:latest, 3391.53MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.2
wandb:   1 of 1 files downloaded.  
wandb: Downloading large artifact ise_classical_supervised_test:latest, 63.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [13]:
# p. 35-38
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type"
]


if subset == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set_extended_60"), engine="fastparquet", columns=columns
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set_extended_20"), engine="fastparquet", columns=columns
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set_extended_20"), engine="fastparquet", columns=columns
    )
    eval_data = pd.concat([train,val,test])
    del train, val, test
    
elif subset == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set_extended_20"), engine="fastparquet", columns=columns
    )

results_data = pd.read_parquet(
    Path(results_dir, "results"), engine="fastparquet"
)

results = []
for i, model in enumerate(models):
    result = pd.read_parquet(
    Path(result_dirs[i], "results"), engine="fastparquet"
    )
    results.append(result)

results_data = pd.concat(results, axis=1)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results

### Robustness Checks

In [15]:
X_print.head()

,buy_sell,EXPIRATION,QUOTE_DATETIME,TRADE_SIZE,TRADE_PRICE,ask_ex,bid_ex,myn,OPTION_TYPE,issue_type
39342171,-1,2015-12-18,2015-11-06 09:30:00,2,0.520000,0.600000,0.520000,0.921659,P,%
39342172,-1,2015-11-27,2015-11-06 09:30:00,1,7.820000,8.150000,7.600000,1.001696,C,0
39342173,-1,2017-01-20,2015-11-06 09:30:00,1,28.889999,32.049999,28.799999,0.886115,C,0
39342174,1,2015-11-20,2015-11-06 09:30:00,1,2.250000,2.250000,1.850000,1.009261,C,%
39342175,-1,2015-12-18,2015-11-06 09:30:00,1,1.700000,1.950000,1.700000,0.991880,C,%


In [17]:

# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map({'0': 'Stock options', 'A': 'Index option', '7': 'Others', 'F':'Others', '%': 'Others', ' ': 'Others'})

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "myn <= 0.7",
    "myn (0.7-0.9]",
    "myn (0.9-1.1]",
    "myn (1.1-1.3]",
    "myn > 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid
mid_ex = 0.5 * (X_print["ask_ex"] + X_print["bid_ex"])
X_print["mid"] = mid_ex

# p. 31
def map_quotes(x):
    if x["TRADE_PRICE"] == x["bid_ex"] or x["TRADE_PRICE"] == x["ask_ex"]:
        return "at quote"
    elif x["TRADE_PRICE"] < x["bid_ex"] or  x["TRADE_PRICE"] > x["ask_ex"]:
        return "outside"
    elif x["TRADE_PRICE"] == x["mid"]:
        return "at mid"
    return "inside"
    # TODO: How to handle unknowns?
    # elif x["bid_ex"] < x["TRADE_PRICE"] < x["mid"]  or x["mid"] < x["TRADE_PRICE"] < x["ask_ex"]:
    #     return "inside"   
    # return "unknown"
    
X_print["prox_q_binned"] = X_print[["TRADE_PRICE", "mid", "bid_ex", "ask_ex"]].apply(map_quotes, axis=1)

# clean up empty buckets, as it causes empty grouping in result set generatio
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()


X_print.drop(columns=["EXPIRATION","QUOTE_DATETIME", "TRADE_SIZE", "ttm", "myn", "ask_ex", "bid_ex", "year"], inplace=True)


In [ ]:
X_print.head(20)

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,mid,prox_q_binned
39342171,-1,0.520000,P,Others,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],0.560000,at quote
39342172,-1,7.820000,C,Stock options,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],7.875000,inside
39342173,-1,28.889999,C,Stock options,"(0,1]",2015,ttm > 12 month,myn (0.7-0.9],30.424999,inside
39342174,1,2.250000,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],2.050000,at quote
39342175,-1,1.700000,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],1.825000,at quote
39342176,1,0.350000,P,Others,"(3,5]",2015,ttm <= 1 month,myn (0.9-1.1],0.280000,inside
39342177,1,0.440000,P,Others,"(3,5]",2015,ttm <= 1 month,myn (0.9-1.1],0.360000,inside
39342178,-1,0.720000,P,Others,"(3,5]",2015,ttm <= 1 month,myn (0.9-1.1],0.810000,inside
39342179,1,11.300000,C,Stock options,"(5,11]",2015,ttm > 12 month,myn > 1.3,NaN,inside
39342180,1,2.190000,P,Stock options,"(1,3]",2015,ttm > 12 month,myn (0.7-0.9],1.980000,inside


In [ ]:
X_print = pd.concat([X_print, results_data], axis=1)

In [ ]:
X_print.head()

,buy_sell,TRADE_PRICE,OPTION_TYPE,issue_type,TRADE_SIZE_binned,year_binned,ttm_binned,myn_binned,mid,prox_q_binned,...,rev_emo(best),clnv(ex),clnv(best),rev_clnv(ex),rev_clnv(best),trade_size(ex)->tick(all),trade_size(ex)->quote(best),trade_size(ex)->quote(best)->quote(ex),quote(best)->quote(ex),trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex)
39342171,-1,0.520000,P,Others,"(1,3]",2015,ttm <= 1 month,myn (0.9-1.1],0.560000,at quote,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
39342172,-1,7.820000,C,Stock options,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],7.875000,inside,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342173,-1,28.889999,C,Stock options,"(0,1]",2015,ttm > 12 month,myn (0.7-0.9],30.424999,inside,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
39342174,1,2.250000,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],2.050000,at quote,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
39342175,-1,1.700000,C,Others,"(0,1]",2015,ttm <= 1 month,myn (0.9-1.1],1.825000,at quote,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## Results Set Generation

In [ ]:
LUT = {"(ex)": " (ex)", 
       "(best)": " (best)",
       "rev_": "Rev. ",
       "tick": "Tick",
       "quote": "Quote",
       "trade_size": "Trade Size",
       "depth": "Depth",
       "->": " $\\to$ ",
       "lr": "\gls{LR}",
       "emo": "\gls{EMO}", 
       "clnv": "\gls{CLNV}",
      "OPTION_TYPE": "Option Type",
      "(":"$(", # put interval start in math env
      "]":"]$", # put interval end in math env
      "_":"$\_"
      }

LUT_INDEX = {"OPTION_TYPE": "Option Type", 
             "issue_type": "Security Type", 
             "TRADE_SIZE_binned": "Trade Size", 
             "year_binned": "Year", 
             "ttm_binned": "Time to Maturity", 
             "myn_binned": "Moneyness",
             "prox_q_binned": "Location to Quote"
}



def cell_str(x):
    for orig, sub in LUT.items():
        x = x.replace(orig,sub)
    return x


def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')


In [ ]:
def set_tex_style(styler, caption, label):
    res = (
    styler.set_caption(caption)
    # .hide(axis="index")
    .apply(highlight_max, props="font-weight:bold;", axis=0) # optionally set subset....    
    .format_index(cell_str, axis = 0)
    .format_index(cell_str, axis = 1)
    .format(precision=4, decimal='.', thousands=",", escape=False, hyperlinks=None)
    .to_latex(f"{label}.tex", siunitx=True, position_float="centering", hrules=True, clines="skip-last;data",
                      label="tab:"+label, caption=caption, convert_css=True)
             )
    return res

In [ ]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)

## Accurcay Calculation

In [29]:
result_dfs = []

for criterion in criterions:
    results = []
    for classifier in classifiers:
        acc_tot = accuracy_score(X_print["buy_sell"], X_print[classifier])
        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(lambda x: accuracy_score(x["buy_sell"], x[classifier]))
            .mul(100)
            .rename(classifier)
            )
        res.loc["all"] = acc_tot * 100
        
        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(set_tex_style, caption=(f"long-tbd", "short-tbd"), label=f"{key.lower()}-{criterion.lower()}")

    # store all result sets for later use
    result_dfs.append(result_df)

Exception ignored in: <function tqdm.__del__ at 0x1551096df3a0>
Traceback (most recent call last):
  File "/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x1551096df3a0>
Traceback (most recent call last):
  File "/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


In [ ]:
result_dfs[0]

Option Type,C,P,all
classical-size,71.871219,72.845956,72.326249
tick(all),53.573900,53.078335,53.342559
tick(ex),50.372016,50.097673,50.243947
quote(best),58.711418,60.582925,59.585081
quote(ex),56.344838,57.740199,56.996225
lr(ex),56.418422,57.801998,57.064307
lr(best),58.774523,60.594004,59.623898
rev_lr(ex),56.486528,57.888561,57.141029
rev_lr(best),58.792914,60.650285,59.659977
emo(ex),53.505205,54.111720,53.788340


In [ ]:
result_dfs[1]

Security Type,Index option,Others,Stock options,all
classical-size,57.421532,76.345621,70.932783,72.326249
tick(all),51.554245,53.257312,53.404570,53.342559
tick(ex),50.871870,49.870976,50.384689,50.243947
quote(best),57.814700,64.962501,57.447408,59.585081
quote(ex),53.720453,62.409791,54.866364,56.996225
lr(ex),53.782831,62.501336,54.925098,57.064307
lr(best),57.820371,64.982376,57.494341,59.623898
rev_lr(ex),53.667527,62.607179,54.993031,57.141029
rev_lr(best),57.820371,65.030275,57.526223,59.659977
emo(ex),51.285832,57.505227,52.329176,53.788340


In [ ]:
result_dfs[2]

Trade Size,"(0,1]","(1,3]","(3,5]","(5,11]",>11,all
classical-size,72.534139,72.950857,72.748349,71.768243,71.297236,72.326249
tick(all),52.848691,52.830617,52.411300,53.630853,55.550232,53.342559
tick(ex),50.289740,50.222920,49.648294,50.186182,50.750727,50.243947
quote(best),58.064698,57.950477,58.330811,62.096424,63.258972,59.585081
quote(ex),55.290135,55.307640,55.736591,59.890120,60.706061,56.996225
lr(ex),55.423031,55.397594,55.774439,59.902085,60.697694,57.064307
lr(best),58.131272,57.984292,58.363067,62.118031,63.271784,59.623898
rev_lr(ex),55.480578,55.452455,55.932575,59.987698,60.765154,57.141029
rev_lr(best),58.176407,58.033288,58.418537,62.148640,63.260737,59.659977
emo(ex),51.933208,51.950476,52.814530,57.157071,57.237072,53.788340


In [ ]:
result_dfs[3]

Year,2015,2016,2017,all
classical-size,69.011751,72.517449,72.998745,72.326249
tick(all),52.788691,53.281666,53.670502,53.342559
tick(ex),50.381045,50.183110,50.339021,50.243947
quote(best),56.015922,59.729647,60.451627,59.585081
quote(ex),54.878241,57.578845,56.356885,56.996225
lr(ex),54.887934,57.639197,56.462584,57.064307
lr(best),56.014491,59.765951,60.509837,59.623898
rev_lr(ex),54.995435,57.719699,56.520163,57.141029
rev_lr(best),56.094015,59.811811,60.508576,59.659977
emo(ex),52.835172,54.268153,52.995354,53.788340


In [ ]:
result_dfs[4]

Time to Maturity,ttm <= 1 month,ttm (1-2] month,ttm (2-3] month,ttm (3-6] month,ttm (6-12] month,ttm > 12 month,all
classical-size,72.732527,72.815921,71.803625,71.275752,71.226768,68.595470,72.326249
tick(all),53.070524,53.404782,53.689628,53.882093,54.278666,54.903416,53.342559
tick(ex),50.124143,50.234919,50.465958,50.823732,50.631747,50.270019,50.243947
quote(best),60.417557,60.309049,59.459459,57.685187,57.463633,50.800908,59.585081
quote(ex),57.307837,57.732111,57.093101,55.978766,56.419706,52.670582,56.996225
lr(ex),57.366438,57.765969,57.177345,56.112714,56.473250,52.850764,57.064307
lr(best),60.438450,60.311838,59.502210,57.790397,57.546824,51.028867,59.623898
rev_lr(ex),57.469937,57.832292,57.172525,56.099023,56.494991,52.866520,57.141029
rev_lr(best),60.505662,60.322195,59.447095,57.750558,57.490944,51.003962,59.659977
emo(ex),54.413938,53.550872,52.892041,52.026183,51.948605,51.190243,53.788340


In [ ]:
result_dfs[5]

Moneyness,myn <= 0.7,myn (0.7-0.9],myn (0.9-1.1],myn (1.1-1.3],myn > 1.3,all
classical-size,71.865108,74.258422,72.949282,66.248435,63.042252,72.326249
tick(all),54.477316,55.496913,52.948661,51.638894,52.034053,53.342559
tick(ex),49.538611,50.675156,50.186938,50.100022,50.042366,50.243947
quote(best),61.259580,63.394870,60.045450,50.023348,48.772913,59.585081
quote(ex),60.365344,60.374359,57.084807,49.931615,48.710672,56.996225
lr(ex),60.428423,60.574909,57.115398,49.995855,48.819439,57.064307
lr(best),61.388021,63.545310,60.040269,50.099607,48.903247,59.623898
rev_lr(ex),60.572848,60.548998,57.214293,50.109692,48.858878,57.141029
rev_lr(best),61.458235,63.472991,60.103392,50.166887,48.882295,59.659977
emo(ex),58.223516,57.563791,53.144117,49.876078,50.041134,53.788340


In [ ]:
result_dfs[6]

Location to Quote,at mid,at quote,inside,outside,all
classical-size,72.120396,86.639010,68.313459,61.383614,72.326249
tick(all),51.026223,57.729733,52.409884,62.376703,53.342559
tick(ex),49.213320,51.380862,50.057898,55.478763,50.243947
quote(best),55.901058,60.206310,59.924963,65.852513,59.585081
quote(ex),49.985180,59.973691,57.138567,66.375897,56.996225
lr(ex),50.625876,59.973793,57.143744,66.375897,57.064307
lr(best),56.239301,60.366086,59.886519,65.812253,59.623898
rev_lr(ex),51.205073,59.973588,57.170209,66.375897,57.141029
rev_lr(best),56.682999,60.228822,59.913159,65.872643,59.659977
emo(ex),51.026223,59.973793,52.409884,62.376703,53.788340


In [ ]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T

In [ ]:
master.iloc[:,0:10]

classical-size  tick(all)   tick(ex)  \
Option Type       C                      71.871219  53.573900  50.372016   
                  P                      72.845956  53.078335  50.097673   
                  all                    72.326249  53.342559  50.243947   
Security Type     Index option           57.421532  51.554245  50.871870   
                  Others                 76.345621  53.257312  49.870976   
                  Stock options          70.932783  53.404570  50.384689   
                  all                    72.326249  53.342559  50.243947   
Trade Size        (0,1]                  72.534139  52.848691  50.289740   
                  (1,3]                  72.950857  52.830617  50.222920   
                  (3,5]                  72.748349  52.411300  49.648294   
                  (5,11]                 71.768243  53.630853  50.186182   
                  >11                    71.297236  55.550232  50.750727   
                  all                    72.326249  53.342559  50.243947   
Year              2015                   69.011751  52.788691  50.381045   
                  2016                   72.517449  53.281666  50.183110   
                  2017                   72.998745  53.670502  50.339021   
                  all                    72.326249  53.342559  50.243947   
Time to Maturity  ttm <= 1 month         72.732527  53.070524  50.124143   
                  ttm (1-2] month        72.815921  53.404782  50.234919   
                  ttm (2-3] month        71.803625  53.689628  50.465958   
                  ttm (3-6] month        71.275752  53.882093  50.823732   
                  ttm (6-12] month       71.226768  54.278666  50.631747   
                  ttm > 12 month         68.595470  54.903416  50.270019   
                  all                    72.326249  53.342559  50.243947   
Moneyness         myn <= 0.7             71.865108  54.477316  49.538611   
                  myn (0.7-0.9]          74.258422  55.496913  50.675156   
                  myn (0.9-1.1]          72.949282  52.948661  50.186938   
                  myn (1.1-1.3]          66.248435  51.638894  50.100022   
                  myn > 1.3              63.042252  52.034053  50.042366   
                  all                    72.326249  53.342559  50.243947   
Location to Quote at mid                 72.120396  51.026223  49.213320   
                  at quote               86.639010  57.729733  51.380862   
                  inside                 68.313459  52.409884  50.057898   
                  outside                61.383614  62.376703  55.478763   
                  all                    72.326249  53.342559  50.243947   

                                    quote(best)  quote(ex)     lr(ex)  \
Option Type       C                   58.711418  56.344838  56.418422   
                  P                   60.582925  57.740199  57.801998   
                  all                 59.585081  56.996225  57.064307   
Security Type     Index option        57.814700  53.720453  53.782831   
                  Others              64.962501  62.409791  62.501336   
                  Stock options       57.447408  54.866364  54.925098   
                  all                 59.585081  56.996225  57.064307   
Trade Size        (0,1]               58.064698  55.290135  55.423031   
                  (1,3]               57.950477  55.307640  55.397594   
                  (3,5]               58.330811  55.736591  55.774439   
                  (5,11]              62.096424  59.890120  59.902085   
                  >11                 63.258972  60.706061  60.697694   
                  all                 59.585081  56.996225  57.064307   
Year              2015                56.015922  54.878241  54.887934   
                  2016                59.729647  57.578845  57.639197   
                  2017                60.451627  56.356885  56.462584   
                  all                 59.585081  56.996225

In [ ]:
master.iloc[:,11:-1]

rev_emo(ex)  rev_emo(best)   clnv(ex)  \
Option Type       C                   53.649405      55.606796  54.339938   
                  P                   54.727324      56.918757  55.220002   
                  all                 54.152602      56.219249  54.750772   
Security Type     Index option        50.218795      50.225410  51.741378   
                  Others              58.159445      60.691923  59.061310   
                  Stock options       52.598969      54.510022  53.060187   
                  all                 54.152602      56.219249  54.750772   
Trade Size        (0,1]               52.443638      54.527170  52.960619   
                  (1,3]               52.531057      54.604379  52.860507   
                  (3,5]               53.643976      55.583435  53.603599   
                  (5,11]              57.563238      59.395087  57.970916   
                  >11                 56.553446      58.948426  58.451558   
                  all                 54.152602      56.219249  54.750772   
Year              2015                53.047751      53.896634  53.310115   
                  2016                54.578223      56.331248  55.220189   
                  2017                53.536533      56.741540  54.146126   
                  all                 54.152602      56.219249  54.750772   
Time to Maturity  ttm <= 1 month      55.105812      57.389778  55.218753   
                  ttm (1-2] month     53.716480      56.123727  54.781831   
                  ttm (2-3] month     52.407956      54.514668  54.139345   
                  ttm (3-6] month     51.367666      52.877975  53.356785   
                  ttm (6-12] month    51.331053      52.287837  53.572892   
                  ttm > 12 month      51.075120      51.284273  52.078701   
                  all                 54.152602      56.219249  54.750772   
Moneyness         myn <= 0.7          58.729289      59.920652  58.963623   
                  myn (0.7-0.9]       57.068770      59.437301  58.610295   
                  myn (0.9-1.1]       53.625367      55.936957  54.333552   
                  myn (1.1-1.3]       51.066807      51.443547  49.670923   
                  myn > 1.3           50.775384      50.924822  48.758431   
                  all                 54.152602      56.219249  54.750772   
Location to Quote at mid              51.833873      56.061055  51.026223   
                  at quote            59.973588      59.587217  59.970985   
                  inside              52.822425      55.280621  53.787043   
                  outside             58.800242      58.343958  62.376703   
                  all                 54.152602      56.219249  54.750772   

                                    clnv(best)  rev_clnv(ex)  rev_clnv(best)  \
Option Type       C                  56.597578     54.463541       56.778181   
                  P                  57.624942     55.789794       58.177595   
                  all                57.077175     55.082666       57.431459   
Security Type     Index option       51.565586     50.663945       50.566598   
                  Others             61.574413     59.616119       62.156604   
                  Stock options      55.350035     53.324088       55.633455   
                  all                57.077175     55.082666       57.431459   
Trade Size        (0,1]              55.325992     53.402249       55.845825   
                  (1,3]              55.227950     53.345081       55.772635   
                  (3,5]              55.942535     54.319270       56.576993   
                  (5,11]             60.052111     58.314762       60.371504   
                  >11                60.905240     58.002671       60.451319   
                  all                57.077175     55.082666       57.431459   
Year              2015               54.346464     53.554966       54.642422   
                  2016               57.213552     55.534174   

In [ ]:
master.style.pipe(set_tex_style, caption=("master-short","master-long"), label=f"{key}-master")

## Effective Spread

In [61]:
classifiers

['classical-size',
 'tick(all)',
 'tick(ex)',
 'quote(best)',
 'quote(ex)',
 'lr(ex)',
 'lr(best)',
 'rev_lr(ex)',
 'rev_lr(best)',
 'emo(ex)',
 'emo(best)',
 'rev_emo(ex)',
 'rev_emo(best)',
 'clnv(ex)',
 'clnv(best)',
 'rev_clnv(ex)',
 'rev_clnv(best)',
 'trade_size(ex)->tick(all)',
 'trade_size(ex)->quote(best)',
 'trade_size(ex)->quote(best)->quote(ex)',
 'quote(best)->quote(ex)',
 'trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex)']

In [88]:
eff_dfs = []

def stats(x, classifier):

    nom = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="nominal")
    rel = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="relative") * 100

    # eff_spread_pred = effective_spread(x[classifier], x["TRADE_PRICE"], x["mid"], mode="none")
    # eff_spread_true = effective_spread(x["buy_sell"], x["TRADE_PRICE"], x["mid"], mode="none")
    # wilcoxon_res  = wilcoxon(eff_spread_pred, eff_spread_true, nan_policy="omit", zero_method="zsplit")
    
    return pd.Series({'nominal':  nom, 
             'rel': rel,
             # 'statistic':wilcoxon_res.statistic,
             # 'pvalue':wilcoxon_res.pvalue
            })

for criterion in criterions:
    results = []
    
    for classifier in classifiers:
        res = (X_print.groupby([criterion])[["TRADE_PRICE", "mid", classifier, "buy_sell"]]
            .apply(stats, classifier)
              )
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1, keys=classifiers).T
    result_df.style.pipe(set_tex_style, caption=(f"long-tbd","short-tbd"), label=f"{key.lower()}-{criterion.lower()}-eff-spread")

    # store all result sets for later use
    eff_dfs.append(result_df)

In [106]:
eff_dfs[0]

OPTION_TYPE                                                         C  \
classical-size                                     nominal   0.045034   
                                                   rel       6.663309   
tick(all)                                          nominal   0.023690   
                                                   rel       2.787034   
tick(ex)                                           nominal   0.016413   
                                                   rel       1.216119   
quote(best)                                        nominal   0.062775   
                                                   rel       8.687065   
quote(ex)                                          nominal   0.171481   
                                                   rel      15.908595   
lr(ex)                                             nominal   0.171481   
                                                   rel      15.908595   
lr(best)                                           nominal   0.063281   
                                                   rel       8.781871   
rev_lr(ex)                                         nominal   0.171481   
                                                   rel      15.908595   
rev_lr(best)                                       nominal   0.062990   
                                                   rel       8.741008   
emo(ex)                                            nominal   0.050400   
                                                   rel       8.211000   
emo(best)                                          nominal   0.047326   
                                                   rel       7.327593   
rev_emo(ex)                                        nominal   0.045780   
                                                   rel       7.694177   
rev_emo(best)                                      nominal   0.043009   
                                                   rel       7.000730   
clnv(ex)                                           nominal   0.122160   
                                                   rel      13.033400   
clnv(best)                                         nominal   0.055455   
                                                   rel       7.958045   
rev_clnv(ex)                                       nominal   0.120710   
                                                   rel      12.910704   
rev_clnv(best)                                     nominal   0.053844   
                                                   rel       7.822577   
trade_size(ex)->tick(all)                          nominal  -0.007608   
                                                   rel       0.157050   
trade_size(ex)->quote(best)                        nominal   0.010157   
                                                   rel       3.987762   
trade_size(ex)->quote(best)->quote(ex)             nominal   0.013925   
                                                   rel       4.309501   
quote(best)->quote(ex)                             nominal   0.067512   
                                                   rel       9.072596   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal   0.013925   
                                                   rel       4.309501   

OPTION_TYPE                                                             P  
classical-size                                     nominal   3.935998e-02  
                                                   rel      9.775967e+302  
tick(all)                                          nominal   1.967647e-02  
                                                   rel      9.775967e+302  
tick(ex)                                           nominal   1.472416e-02  
                                                   rel      9.775967e+302  
quote(best)                                        nominal   5.130346e-02  
                                                   rel      9.775967e+302  
quote(ex)                     

In [107]:
eff_dfs[1]

issue_type                                                  Index option  \
classical-size                                     nominal      0.128587   
                                                   rel          0.602381   
tick(all)                                          nominal      0.034653   
                                                   rel          0.705226   
tick(ex)                                           nominal      0.024063   
                                                   rel          0.602702   
quote(best)                                        nominal      0.102625   
                                                   rel          2.356139   
quote(ex)                                          nominal      0.563742   
                                                   rel          7.288216   
lr(ex)                                             nominal      0.563742   
                                                   rel          7.288216   
lr(best)                                           nominal      0.104647   
                                                   rel          2.367328   
rev_lr(ex)                                         nominal      0.563742   
                                                   rel          7.288216   
rev_lr(best)                                       nominal      0.104487   
                                                   rel          2.356349   
emo(ex)                                            nominal      0.060771   
                                                   rel          1.724806   
emo(best)                                          nominal      0.060412   
                                                   rel          1.712909   
rev_emo(ex)                                        nominal      0.055739   
                                                   rel          1.212132   
rev_emo(best)                                      nominal      0.055242   
                                                   rel          1.217619   
clnv(ex)                                           nominal      0.181988   
                                                   rel          4.133225   
clnv(best)                                         nominal      0.066934   
                                                   rel          1.702426   
rev_clnv(ex)                                       nominal      0.188071   
                                                   rel          4.160702   
rev_clnv(best)                                     nominal      0.070783   
                                                   rel          1.307506   
trade_size(ex)->tick(all)                          nominal      0.024238   
                                                   rel          0.587715   
trade_size(ex)->quote(best)                        nominal      0.071541   
                                                   rel          1.651301   
trade_size(ex)->quote(best)->quote(ex)             nominal      0.085599   
                                                   rel          1.746990   
quote(best)->quote(ex)                             nominal      0.122452   
                                                   rel          2.479553   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal      0.085599   
                                                   rel          1.746990   

issue_type                                                     Others  \
classical-size                                     nominal   0.030540   
                                                   rel      10.922618   
tick(all)                                          nominal   0.013705   
                                                   rel       0.996820   
tick(ex)                                           nominal   0.008241   
                                                   rel      -0.785548   
quote(best)                                        nominal   0.04259

In [108]:
eff_dfs[2]

TRADE_SIZE_binned                                               (0,1]  \
classical-size                                     nominal   0.038047   
                                                   rel       4.853544   
tick(all)                                          nominal   0.025029   
                                                   rel       2.853949   
tick(ex)                                           nominal   0.017960   
                                                   rel       1.615596   
quote(best)                                        nominal   0.066373   
                                                   rel       7.540989   
quote(ex)                                          nominal   0.187541   
                                                   rel      14.934810   
lr(ex)                                             nominal   0.187541   
                                                   rel      14.934810   
lr(best)                                           nominal   0.066955   
                                                   rel       7.641033   
rev_lr(ex)                                         nominal   0.187541   
                                                   rel      14.934810   
rev_lr(best)                                       nominal   0.066600   
                                                   rel       7.601823   
emo(ex)                                            nominal   0.051456   
                                                   rel       7.015676   
emo(best)                                          nominal   0.048019   
                                                   rel       6.138223   
rev_emo(ex)                                        nominal   0.045795   
                                                   rel       6.432262   
rev_emo(best)                                      nominal   0.042839   
                                                   rel       5.747699   
clnv(ex)                                           nominal   0.128571   
                                                   rel      11.931760   
clnv(best)                                         nominal   0.057030   
                                                   rel       6.779406   
rev_clnv(ex)                                       nominal   0.126906   
                                                   rel      11.800950   
rev_clnv(best)                                     nominal   0.054783   
                                                   rel       6.613555   
trade_size(ex)->tick(all)                          nominal  -0.007324   
                                                   rel       1.062228   
trade_size(ex)->quote(best)                        nominal   0.012231   
                                                   rel       4.334164   
trade_size(ex)->quote(best)->quote(ex)             nominal   0.016430   
                                                   rel       4.672034   
quote(best)->quote(ex)                             nominal   0.071797   
                                                   rel       7.944337   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal   0.016430   
                                                   rel       4.672034   

TRADE_SIZE_binned                                               (1,3]  \
classical-size                                     nominal   0.042270   
                                                   rel       5.728606   
tick(all)                                          nominal   0.022638   
                                                   rel       2.633597   
tick(ex)                                           nominal   0.015399   
                                                   rel       1.345637   
quote(best)                                        nominal   0.058722   
                                                   rel       8.071728   
quote(ex)                                          nomina

In [109]:
eff_dfs[3]

year_binned                                                      2015  \
classical-size                                     nominal   0.049976   
                                                   rel       6.164320   
tick(all)                                          nominal   0.016895   
                                                   rel       2.362699   
tick(ex)                                           nominal   0.011205   
                                                   rel       1.274827   
quote(best)                                        nominal   0.061677   
                                                   rel       8.207709   
quote(ex)                                          nominal   0.147433   
                                                   rel      13.745120   
lr(ex)                                             nominal   0.147433   
                                                   rel      13.745120   
lr(best)                                           nominal   0.061686   
                                                   rel       8.300404   
rev_lr(ex)                                         nominal   0.147433   
                                                   rel      13.745120   
rev_lr(best)                                       nominal   0.061751   
                                                   rel       8.267998   
emo(ex)                                            nominal   0.037976   
                                                   rel       7.063606   
emo(best)                                          nominal   0.034886   
                                                   rel       6.342800   
rev_emo(ex)                                        nominal   0.037483   
                                                   rel       6.541553   
rev_emo(best)                                      nominal   0.035325   
                                                   rel       6.012196   
clnv(ex)                                           nominal   0.093622   
                                                   rel      10.890067   
clnv(best)                                         nominal   0.044309   
                                                   rel       7.060379   
rev_clnv(ex)                                       nominal   0.094616   
                                                   rel      10.762380   
rev_clnv(best)                                     nominal   0.047663   
                                                   rel       6.967438   
trade_size(ex)->tick(all)                          nominal  -0.006514   
                                                   rel      -0.128030   
trade_size(ex)->quote(best)                        nominal   0.019842   
                                                   rel       3.780892   
trade_size(ex)->quote(best)->quote(ex)             nominal   0.022534   
                                                   rel       4.030468   
quote(best)->quote(ex)                             nominal   0.064797   
                                                   rel       8.488241   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal   0.022534   
                                                   rel       4.030468   

year_binned                                                          2016  \
classical-size                                     nominal   4.309619e-02   
                                                   rel      3.596854e+302   
tick(all)                                          nominal   2.227762e-02   
                                                   rel      3.596854e+302   
tick(ex)                                           nominal   1.526681e-02   
                                                   rel      3.596854e+302   
quote(best)                                        nominal   5.713752e-02   
                                                   rel      3.596854e+302   
quote(ex)            

In [110]:
eff_dfs[4]

ttm_binned                                                  ttm <= 1 month  \
classical-size                                     nominal        0.032901   
                                                   rel            8.813669   
tick(all)                                          nominal        0.019678   
                                                   rel            2.947513   
tick(ex)                                           nominal        0.017074   
                                                   rel            1.344642   
quote(best)                                        nominal        0.045934   
                                                   rel           11.231114   
quote(ex)                                          nominal        0.147437   
                                                   rel           19.914330   
lr(ex)                                             nominal        0.147437   
                                                   rel           19.914330   
lr(best)                                           nominal        0.046479   
                                                   rel           11.368463   
rev_lr(ex)                                         nominal        0.147437   
                                                   rel           19.914330   
rev_lr(best)                                       nominal        0.046158   
                                                   rel           11.306912   
emo(ex)                                            nominal        0.039406   
                                                   rel           10.669335   
emo(best)                                          nominal        0.035401   
                                                   rel            9.464541   
rev_emo(ex)                                        nominal        0.031998   
                                                   rel            9.763649   
rev_emo(best)                                      nominal        0.028888   
                                                   rel            8.849703   
clnv(ex)                                           nominal        0.105701   
                                                   rel           16.387110   
clnv(best)                                         nominal        0.041346   
                                                   rel           10.259345   
rev_clnv(ex)                                       nominal        0.104286   
                                                   rel           16.205704   
rev_clnv(best)                                     nominal        0.037813   
                                                   rel            9.968051   
trade_size(ex)->tick(all)                          nominal       -0.003555   
                                                   rel           -0.088008   
trade_size(ex)->quote(best)                        nominal        0.007491   
                                                   rel            5.417915   
trade_size(ex)->quote(best)->quote(ex)             nominal        0.011210   
                                                   rel            5.836436   
quote(best)->quote(ex)                             nominal        0.050529   
                                                   rel           11.720164   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal        0.011210   
                                                   rel            5.836436   

ttm_binned                                                  ttm (1-2] month  \
classical-size                                     nominal         0.044063   
                                                   rel             4.057045   
tick(all)                                          nominal         0.017443   
                                                   rel             1.870065   
tick(ex)                                           nominal         0.009775   
               

In [111]:
eff_dfs[5]

myn_binned                                                  myn <= 0.7  \
classical-size                                     nominal    0.019004   
                                                   rel       13.281575   
tick(all)                                          nominal    0.038055   
                                                   rel        6.737995   
tick(ex)                                           nominal    0.034122   
                                                   rel        2.995119   
quote(best)                                        nominal    0.064909   
                                                   rel       21.297551   
quote(ex)                                          nominal    0.201324   
                                                   rel       34.937756   
lr(ex)                                             nominal    0.201324   
                                                   rel       34.937756   
lr(best)                                           nominal    0.066166   
                                                   rel       21.569765   
rev_lr(ex)                                         nominal    0.201324   
                                                   rel       34.937756   
rev_lr(best)                                       nominal    0.065337   
                                                   rel       21.438545   
emo(ex)                                            nominal    0.075594   
                                                   rel       20.492897   
emo(best)                                          nominal    0.068098   
                                                   rel       18.739349   
rev_emo(ex)                                        nominal    0.064793   
                                                   rel       18.762847   
rev_emo(best)                                      nominal    0.058884   
                                                   rel       17.359188   
clnv(ex)                                           nominal    0.172056   
                                                   rel       29.807850   
clnv(best)                                         nominal    0.070952   
                                                   rel       20.094494   
rev_clnv(ex)                                       nominal    0.172519   
                                                   rel       29.544659   
rev_clnv(best)                                     nominal    0.065161   
                                                   rel       19.344482   
trade_size(ex)->tick(all)                          nominal   -0.003114   
                                                   rel       -1.070673   
trade_size(ex)->quote(best)                        nominal   -0.002478   
                                                   rel        6.904063   
trade_size(ex)->quote(best)->quote(ex)             nominal    0.001813   
                                                   rel        7.576810   
quote(best)->quote(ex)                             nominal    0.069837   
                                                   rel       22.136974   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal    0.001813   
                                                   rel        7.576810   

myn_binned                                                  myn (0.7-0.9]  \
classical-size                                     nominal       0.027250   
                                                   rel          13.418302   
tick(all)                                          nominal       0.024780   
                                                   rel           4.708344   
tick(ex)                                           nominal       0.022565   
                                                   rel           2.348643   
quote(best)                                        nominal       0.043103   
                                                   re

In [112]:
eff_dfs[6]

prox_q_binned                                               at mid   at quote  \
classical-size                                     nominal     0.0  -0.025131   
                                                   rel         0.0  17.109239   
tick(all)                                          nominal     0.0   0.039821   
                                                   rel         0.0   2.462868   
tick(ex)                                           nominal     0.0   0.021083   
                                                   rel         0.0  -3.655351   
quote(best)                                        nominal     0.0   0.159269   
                                                   rel         0.0  31.905514   
quote(ex)                                          nominal     0.0   0.164077   
                                                   rel         0.0  32.362553   
lr(ex)                                             nominal     0.0   0.164077   
                                                   rel         0.0  32.362553   
lr(best)                                           nominal     0.0   0.159432   
                                                   rel         0.0  31.882354   
rev_lr(ex)                                         nominal     0.0   0.164077   
                                                   rel         0.0  32.362553   
rev_lr(best)                                       nominal     0.0   0.159083   
                                                   rel         0.0  31.878199   
emo(ex)                                            nominal     0.0   0.164077   
                                                   rel         0.0  32.362553   
emo(best)                                          nominal     0.0   0.157638   
                                                   rel         0.0  31.464631   
rev_emo(ex)                                        nominal     0.0   0.164077   
                                                   rel         0.0  32.362553   
rev_emo(best)                                      nominal     0.0   0.156987   
                                                   rel         0.0  31.472508   
clnv(ex)                                           nominal     0.0   0.163885   
                                                   rel         0.0  32.345538   
clnv(best)                                         nominal     0.0   0.157848   
                                                   rel         0.0  31.478638   
rev_clnv(ex)                                       nominal     0.0   0.163936   
                                                   rel         0.0  32.352133   
rev_clnv(best)                                     nominal     0.0   0.157277   
                                                   rel         0.0  31.495906   
trade_size(ex)->tick(all)                          nominal     0.0  -0.072074   
                                                   rel         0.0  -9.935807   
trade_size(ex)->quote(best)                        nominal     0.0  -0.034921   
                                                   rel         0.0   7.820090   
trade_size(ex)->quote(best)->quote(ex)             nominal     0.0  -0.033451   
                                                   rel         0.0   7.950622   
quote(best)->quote(ex)                             nominal     0.0   0.162869   
                                                   rel         0.0  32.267394   
trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex) nominal     0.0  -0.033451   
                                                   rel         0.0   7.950622   

prox_q_binned                                                  inside  \
classical-size                                     nominal   0.067530   
                                                   rel       5.033986   
tick(all)                                          nominal   0.018993   
                                                   rel       2

In [113]:
master = pd.concat(eff_dfs, axis=1, keys = LUT_INDEX.values()).T

In [114]:
master

classical-size                tick(all)  \
                                          nominal            rel   nominal   
Option Type       C                      0.045034   6.663309e+00  0.023690   
                  P                      0.039360  9.775967e+302  0.019676   
Security Type     Index option           0.128587   6.023815e-01  0.034653   
                  Others                 0.030540   1.092262e+01  0.013705   
                  Stock options          0.045819   5.448746e+00  0.024881   
Trade Size        (0,1]                  0.038047   4.853544e+00  0.025029   
                  (1,3]                  0.042270   5.728606e+00  0.022638   
                  (3,5]                  0.040207   5.859333e+00  0.019875   
                  (5,11]                 0.043894   9.884575e+00  0.019467   
                  >11                    0.051746   1.064610e+01  0.018281   
Year              2015                   0.049976   6.164320e+00  0.016895   
                  2016                   0.043096  3.596854e+302  0.022278   
                  2017                   0.038179   7.150329e+00  0.022402   
Time to Maturity  ttm <= 1 month         0.032901   8.813669e+00  0.019678   
                  ttm (1-2] month        0.044063   4.057045e+00  0.017443   
                  ttm (2-3] month        0.051212   3.140338e+00  0.021077   
                  ttm (3-6] month        0.063586   3.015841e+00  0.027972   
                  ttm (6-12] month       0.067209   2.466502e+00  0.028432   
                  ttm > 12 month         0.108283   2.077753e+00  0.047843   
Moneyness         myn <= 0.7             0.019004   1.328158e+01  0.038055   
                  myn (0.7-0.9]          0.027250   1.341830e+01  0.024780   
                  myn (0.9-1.1]          0.041359  3.389003e+302  0.018608   
                  myn (1.1-1.3]          0.075467   9.273605e-01  0.028592   
                  myn > 1.3              0.098973   9.515554e-01  0.038125   
Location to Quote at mid                 0.000000   0.000000e+00  0.000000   
                  at quote              -0.025131   1.710924e+01  0.039821   
                  inside                 0.067530   5.033986e+00  0.018993   
                  outside                0.128042  -6.711295e+05  0.410488   

                                                   tick(ex)                 \
                                              rel   nominal            rel   
Option Type       C                  2.787034e+00  0.016413   1.216119e+00   
                  P                 9.775967e+302  0.014724  9.775967e+302   
Security Type     Index option       7.052260e-01  0.024063   6.027022e-01   
                  Others             9.968202e-01  0.008241  -7.855478e-01   
                  Stock options      3.132711e+00  0.018466   1.842353e+00   
Trade Size        (0,1]              2.853949e+00  0.017960   1.615596e+00   
                  (1,3]              2.633597e+00  0.015399   1.345637e+00   
                  (3,5]              2.188958e+00  0.015507   9.343630e-01   
                  (5,11]             2.045184e+00  0.014679   5.103243e-01   
                  >11                2.326401e+00  0.012231   3.544626e-01   
Year              2015               2.362699e+00  0.011205   1.274827e+00   
                  2016              3.596854e+302  0.015267  3.596854e+302   
                  2017               3.008716e+00  0.017943   1.450287e+00   
Time to Maturity  ttm <= 1 month     2.947513e+00  0.017074   1.344642e+00   
                  ttm (1-2] month    1.870065e+00  0.009775   7.218353e-01   
                  ttm (2-3] month    1.636938e+00  0.011682   5.741593e-01   
                  ttm (3-6] month    1.593565e+00  0.014216   5.561117e-01   
                  ttm (6-12] month   1.329861e+00  0.012935   4.539161e-01   
                  ttm > 12 month     1.120096e+00  0.017647   1.566632e-01   
Moneyness         myn <= 0.7         6.737995e+00  0.034

In [115]:
master.iloc[:,0:20]

classical-size                tick(all)  \
                                          nominal            rel   nominal   
Option Type       C                      0.045034   6.663309e+00  0.023690   
                  P                      0.039360  9.775967e+302  0.019676   
Security Type     Index option           0.128587   6.023815e-01  0.034653   
                  Others                 0.030540   1.092262e+01  0.013705   
                  Stock options          0.045819   5.448746e+00  0.024881   
Trade Size        (0,1]                  0.038047   4.853544e+00  0.025029   
                  (1,3]                  0.042270   5.728606e+00  0.022638   
                  (3,5]                  0.040207   5.859333e+00  0.019875   
                  (5,11]                 0.043894   9.884575e+00  0.019467   
                  >11                    0.051746   1.064610e+01  0.018281   
Year              2015                   0.049976   6.164320e+00  0.016895   
                  2016                   0.043096  3.596854e+302  0.022278   
                  2017                   0.038179   7.150329e+00  0.022402   
Time to Maturity  ttm <= 1 month         0.032901   8.813669e+00  0.019678   
                  ttm (1-2] month        0.044063   4.057045e+00  0.017443   
                  ttm (2-3] month        0.051212   3.140338e+00  0.021077   
                  ttm (3-6] month        0.063586   3.015841e+00  0.027972   
                  ttm (6-12] month       0.067209   2.466502e+00  0.028432   
                  ttm > 12 month         0.108283   2.077753e+00  0.047843   
Moneyness         myn <= 0.7             0.019004   1.328158e+01  0.038055   
                  myn (0.7-0.9]          0.027250   1.341830e+01  0.024780   
                  myn (0.9-1.1]          0.041359  3.389003e+302  0.018608   
                  myn (1.1-1.3]          0.075467   9.273605e-01  0.028592   
                  myn > 1.3              0.098973   9.515554e-01  0.038125   
Location to Quote at mid                 0.000000   0.000000e+00  0.000000   
                  at quote              -0.025131   1.710924e+01  0.039821   
                  inside                 0.067530   5.033986e+00  0.018993   
                  outside                0.128042  -6.711295e+05  0.410488   

                                                   tick(ex)                 \
                                              rel   nominal            rel   
Option Type       C                  2.787034e+00  0.016413   1.216119e+00   
                  P                 9.775967e+302  0.014724  9.775967e+302   
Security Type     Index option       7.052260e-01  0.024063   6.027022e-01   
                  Others             9.968202e-01  0.008241  -7.855478e-01   
                  Stock options      3.132711e+00  0.018466   1.842353e+00   
Trade Size        (0,1]              2.853949e+00  0.017960   1.615596e+00   
                  (1,3]              2.633597e+00  0.015399   1.345637e+00   
                  (3,5]              2.188958e+00  0.015507   9.343630e-01   
                  (5,11]             2.045184e+00  0.014679   5.103243e-01   
                  >11                2.326401e+00  0.012231   3.544626e-01   
Year              2015               2.362699e+00  0.011205   1.274827e+00   
                  2016              3.596854e+302  0.015267  3.596854e+302   
                  2017               3.008716e+00  0.017943   1.450287e+00   
Time to Maturity  ttm <= 1 month     2.947513e+00  0.017074   1.344642e+00   
                  ttm (1-2] month    1.870065e+00  0.009775   7.218353e-01   
                  ttm (2-3] month    1.636938e+00  0.011682   5.741593e-01   
                  ttm (3-6] month    1.593565e+00  0.014216   5.561117e-01   
                  ttm (6-12] month   1.329861e+00  0.012935   4.539161e-01   
                  ttm > 12 month     1.120096e+00  0.017647   1.566632e-01   
Moneyness         myn <= 0.7         6.737995e+00  0.034

In [116]:
master.iloc[:,21:40]

emo(best) rev_emo(ex)                 \
                                              rel     nominal            rel   
Option Type       C                  7.327593e+00    0.045780   7.694177e+00   
                  P                 9.775967e+302    0.036976  9.775967e+302   
Security Type     Index option       1.712909e+00    0.055739   1.212132e+00   
                  Others             1.127714e+01    0.029192   1.150720e+01   
                  Stock options      6.180699e+00    0.046446   6.527171e+00   
Trade Size        (0,1]              6.138223e+00    0.045795   6.432262e+00   
                  (1,3]              6.611263e+00    0.041968   6.910765e+00   
                  (3,5]              6.965020e+00    0.039507   7.286308e+00   
                  (5,11]             1.007952e+01    0.040073   1.037978e+01   
                  >11                9.598867e+00    0.036302   9.965578e+00   
Year              2015               6.342800e+00    0.037483   6.541553e+00   
                  2016              3.596854e+302    0.039563  3.596854e+302   
                  2017               7.321249e+00    0.047970   7.871146e+00   
Time to Maturity  ttm <= 1 month     9.464541e+00    0.031998   9.763649e+00   
                  ttm (1-2] month    4.266646e+00    0.039087   4.637174e+00   
                  ttm (2-3] month    3.747668e+00    0.049468   4.009266e+00   
                  ttm (3-6] month    3.798911e+00    0.063727   4.178946e+00   
                  ttm (6-12] month   3.271678e+00    0.072864   3.546218e+00   
                  ttm > 12 month     2.772001e+00    0.112080   3.051577e+00   
Moneyness         myn <= 0.7         1.873935e+01    0.064793   1.876285e+01   
                  myn (0.7-0.9]      1.504536e+01    0.041260   1.539490e+01   
                  myn (0.9-1.1]     3.389003e+302    0.037152  3.389003e+302   
                  myn (1.1-1.3]      1.137427e+00    0.056010   1.090225e+00   
                  myn > 1.3          1.184167e+00    0.078695   1.274447e+00   
Location to Quote at mid             0.000000e+00    0.000000   0.000000e+00   
                  at quote           3.146463e+01    0.164077   3.236255e+01   
                  inside             1.834274e+00    0.013038   2.045693e+00   
                  outside           -6.711258e+05   -0.055120  -6.711364e+05   

                                   rev_emo(best)                 clnv(ex)  \
                                         nominal            rel   nominal   
Option Type       C                     0.043009   7.000730e+00  0.122160   
                  P                     0.034225  9.775967e+302  0.109473   
Security Type     Index option          0.055242   1.217619e+00  0.181988   
                  Others                0.027514   1.108161e+01  0.077548   
                  Stock options         0.043213   5.740075e+00  0.130772   
Trade Size        (0,1]                 0.042839   5.747699e+00  0.128571   
                  (1,3]                 0.039394   6.215395e+00  0.116847   
                  (3,5]                 0.037071   6.621546e+00  0.113159   
                  (5,11]                0.036800   9.699466e+00  0.109913   
                  >11                   0.033982   9.334917e+00  0.099423   
Year              2015                  0.035325   6.012196e+00  0.093622   
                  2016                  0.036933  3.596854e+302  0.110366   
                  2017                  0.044699   6.980286e+00  0.137475   
Time to Maturity  ttm <= 1 month        0.028888   8.849703e+00  0.105701   
                  ttm (1-2] month       0.037489   4.410013e+00  0.099833   
                  ttm (2-3] month       0.047751   3.796441e+00  0.113087   
                  ttm (3-6] month       0.061430   4.001738e+00  0.142432   
                  ttm (6-12] month      0.070491   3.371787e+00  0.161956   
                  ttm > 12 month        0.109436   2.934640e+00  0.220692   
Moneyness         myn 

In [117]:
master.iloc[:,41:-1]

quote(best)->quote(ex)  \
                                                      rel   
Option Type       C                          9.072596e+00   
                  P                         9.775967e+302   
Security Type     Index option               2.479553e+00   
                  Others                     1.312378e+01   
                  Stock options              7.997564e+00   
Trade Size        (0,1]                      7.944337e+00   
                  (1,3]                      8.495901e+00   
                  (3,5]                      8.843360e+00   
                  (5,11]                     1.188705e+01   
                  >11                        1.131866e+01   
Year              2015                       8.488241e+00   
                  2016                      3.596854e+302   
                  2017                       9.077348e+00   
Time to Maturity  ttm <= 1 month             1.172016e+01   
                  ttm (1-2] month            5.510656e+00   
                  ttm (2-3] month            4.645054e+00   
                  ttm (3-6] month            4.685371e+00   
                  ttm (6-12] month           3.872190e+00   
                  ttm > 12 month             3.463822e+00   
Moneyness         myn <= 0.7                 2.213697e+01   
                  myn (0.7-0.9]              1.834725e+01   
                  myn (0.9-1.1]             3.389003e+302   
                  myn (1.1-1.3]              2.000534e+00   
                  myn > 1.3                  2.101610e+00   
Location to Quote at mid                     0.000000e+00   
                  at quote                   3.226739e+01   
                  inside                     4.179043e+00   
                  outside                   -6.711138e+05   

                                   trade_size(ex)->depth(ex)->quote(best)->rev_lr(ex)  
                                                                              nominal  
Option Type       C                                                          0.013925  
                  P                                                          0.013647  
Security Type     Index option                                               0.085599  
                  Others                                                     0.010012  
                  Stock options                                              0.014200  
Trade Size        (0,1]                                                      0.016430  
                  (1,3]                                                      0.017759  
                  (3,5]                                                      0.011428  
                  (5,11]                                                     0.010799  
                  >11                                                        0.008216  
Year              2015                                                       0.022534  
                  2016                                                       0.017216  
                  2017                                                       0.002914  
Time to Maturity  ttm <= 1 month                                             0.011210  
                  ttm (1-2] month                                            0.012926  
                  ttm (2-3] month                                            0.014091  
                  ttm (3-6] month                                            0.020385  
                  ttm (6-12] month                                           0.017794  
                  ttm > 12 month                                             0.039943  
Moneyness         myn <= 0.7                                                 0.001813  
                  myn (0.7-0.9]                                              0.007041  
                  myn (0.9-1.1]                                              0.012341  
                  myn (1.1-1.3]                                              0.039167  
    

In [118]:
master.style.pipe(set_tex_style, caption=("master-short","master-long"), label=f"{key}-master-eff-spread")

## Change in Parenthesis

```latex
# https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290
\begin{table}
    \centering
    \caption{test of combination with change}
    \label{tab:combo}
    \begin{tabular}{lSSSSSSSS}
        \toprule
        {} & \multicolumn{2}{l}{Index option} & \multicolumn{2}{l}{Others} & \multicolumn{2}{l}{Stock options} & \multicolumn{2}{l}{all} \\
        \midrule
        classical-size & 1.0 & \parl-56.42\parr & 2.0 & \parl-74.35\parr & -73.5 & \parl-143.93\parr & 5.0 & \parl-67.33\parr \\
        \bottomrule
        \end{tabular}
\end{table}
```

In [119]:
foo = pd.DataFrame([[4,3 ,8, 5]])
bar = pd.DataFrame([[1.1,2 ,73, 5]], columns=foo.columns, index=foo.index)

In [120]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.
    
    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290
    
    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])
    
    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)
    
    for i, mul_col in enumerate(combo.columns):
        
        # define custom brackets that are not parsed by sunitx
        combo[[(mul_col[0], "pm")]] = (
            "\parl" + (combo[mul_col] - base[mul_col[0]]).round(2).astype(str) + "\parr"
            )
        # sort to group together columns
        combo.sort_index(axis=1, inplace=True)
    return combo

In [121]:
combo = combine_results(bar, foo)

# manually replace S with S[table-format=1.4(5)] if needed
combo.style.to_latex(f"combo.tex", siunitx=True, position_float="centering", hrules=True, clines="skip-last;data", label="tab:combo", caption="test of combination with change", multicol_align="l")
             

In [122]:
combo

0                    1                     2                    3  \
   nom              pm  nom              pm   nom              pm  nom   
0  1.1  \parl-2.9\parr  2.0  \parl-1.0\parr  73.0  \parl65.0\parr  5.0   

                  
              pm  
0  \parl0.0\parr